In [71]:
import json
import csv
import pandas as pd
import glob
import os
import gzip
import re
import math
import multiprocessing as mp

In [72]:
filename = "/data/censorship/gfwatch_censored_domains.csv"
df = pd.read_csv(filename)
list_ = []
index = df.columns.tolist()
column_names = index[0].split("|")
for i in df[df.columns[0]]:
    list_.append(i.split("|"))
gfwatch = pd.DataFrame(list_, columns = column_names)
gfwatch




    
    
        
    
    


,Tranco_rank,censored_domain,base_censored_domain,blocking_rules,first_seen,last_seen,base_domain_category
0,88888888,'ioggio95bello.itload.info,itload.info,*itload.info,2022/01/29,2022/01/29,entertainment
1,88888888,----------0----------.blogspot.com,blogspot.com,*.blogspot.com,2021/04/13,2021/11/07,personal websites and blogs
2,88888888,------2-.tumblr.com,tumblr.com,*.tumblr.com,2021/11/16,2021/12/20,personal websites and blogs
3,88888888,-----.deviantart.com,deviantart.com,*deviantart.com,2021/09/20,2021/09/20,arts and culture
4,88888888,----lukyrh.blogspot.com,blogspot.com,*.blogspot.com,2021/12/16,2021/12/16,personal websites and blogs
...,...,...,...,...,...,...,...
3486299,88888888,zzzzzzzv09.in.net,zzzzzzzv09.in.net,zzzzzzzv09.in.net,2021/06/09,2021/06/09,web hosting
3486300,88888888,zzzzzzzz-dot-kinozal-tv.appspot.com,appspot.com,*appspot.com,2021/08/28,2022/02/14,information technology
3486301,88888888,zzzzzzzzz.myftp.org,myftp.org,*.myftp.org*,2021/05/30,2022/02/17,dynamic dns
3486302,88888888,~.ambertovenezia.blogspot.it,blogspot.it,*blogspot.it,2022/02/11,2022/02/12,personal websites and blogs


In [74]:
def process_date(time):
    converted_datetime = pd.to_datetime(arg=time, format='%Y/%m/%d')

    return converted_datetime

def process_time(time):
    converted_datetime = pd.to_datetime(arg=time, format='%Y-%m-%d %H:%M:%S')

    return converted_datetime
def translate_to_regex(rule):
    return_str = rule
    if return_str[0]=="*":
        return_str = ".*"+return_str[1:]
        
    if return_str[-1]=="*":
        return_str = return_str[:-1]+".*"
    return return_str
def match_rule(domain, rule):
    
    if re.match(rule,domain):
        return True
    return False

def find_match_rule(domain):
    for dic_key in dic_censored.keys():
        if dic_key in domain:
            return dic_key
    return ""

def check_domain_block(domain):
    return domain in censored_domains.keys()

def check_range_time(start_time, range_times):
    for i in range(len(range_times)):
        range_time = range_times[i]
        start = process_date(range_time[0])
        end = process_date(range_time[1])
        status = (start_time.date()>= start) and (start_time.date() <= end)
        if status:
            return "Confirmed"
    return "Possible"
    

def label_gwatch(df):
#     censored = list(gfwatch["base_censored_domain"])
    dns_blocking = []
    
    for index, row in df.iterrows():
        time_ranges=[]
        domain = row["input"]
        if check_domain_block(domain):
            time_ranges = censored_domains[domain]
        else:
            matched_domain = find_match_rule(domain)
            if matched_domain !="":
                blocking_rule = dic_censored[matched_domain]["blocking rule"]
                
                if match_rule(domain, blocking_rule):
                    time_ranges= dic_censored[matched_domain]["blocking time"]
        if len(time_ranges) >0:
                dns_blocking.append(check_range_time(row["measurement_start_time"],time_ranges))
                print("censored !")
            
        else:
                print(" Not matched pattern!")
                print(domain)
                dns_blocking.append("")
    df["GFWatchblocking_truth"]=dns_blocking
    return df
                                    
def generate_dates(start_date, end_date):
    lst =  pd.date_range(start_date, end_date, freq='D')
    
    list_date = []
    for i in range(len(lst)):
        list_date.append(lst[i].date().strftime("%Y-%m-%d"))
    return list_date


def process_asn(asn):
    return asn[2:]
def get_domainname(name):
    new_name = name.split("//")[-1]
#     if new_name[:4]=="www.":
#         new_name = new_name[4:]
    if new_name[-1]=="/":
        new_name = new_name[:-1]
    return new_name
# def match_block_signature(ip):
#     blocked = False
#     block_signature = [ "8.7.198.45","37.61.54.158", "46.82.174.68","78.16.49.15","93.46.8.89","159.106.121.75","203.98.7.65", "59.24.3.173","203.98.7.65", "243.185.187.39"]

#     ip_ = ip.split(":")[-1]
# #     print(type(ip_))
# #     print(ip_)
#     if ip.split(":")[-1] in block_signature:
#         blocked = True
#     return blocked

# def label_groundtruth(df):
    
# #     df['dns_blocking_truth'] = [match_block_signature(x) for x in df["test_keys_ipv4"]]
# #     block_level = []
#     for i in df['dns_blocking_truth']:
#         if i:
#             block_level.append("Country")
#         else:
#             block_level.append("")
#     df["Block_level"] = block_level     
#     return df

def process_dataframe(df):
    print("starting")
    df["input"] = [get_domainname(name) for name in df["input"]]
    for col in df.columns:
        if col=="probe_asn" or col=="resolver_asn":
            df[col]=process_asn(df[col])
        elif col == "measurement_start_time" or col == "test_start_time":
            df[col] = process_time(df[col])
#     df = label_groundtruth(df)

    df = label_gwatch(df)
    return df

def split_df(df, splits):
    split_size = math.ceil(df.shape[0]/splits)
    df_list = []
    for i in range(splits):
        if i!= splits-1:
            df_ = df.iloc[list(range(i*split_size,(i+1)*split_size))]
            df_list.append(df_)
        else:
            df_ = df.iloc[list(range(i*split_size,df.shape[0]))]
            df_list.append(df_)
            
    return df_list
            

In [75]:
dic_censored = {}
censored_domains = {}
for index, row in gfwatch.iterrows():
    domain = row["censored_domain"]
    based_censored_domain = row["base_censored_domain"]
    blocking_rule = row["blocking_rules"]
    start = row["first_seen"]
    end = row["last_seen"]
    blocking_rule = translate_to_regex(blocking_rule)
    
    if domain not in censored_domains.keys():
        censored_domains[domain]=[]
    sub_dic = censored_domains[domain]
    sub_dic.append((start, end))
          
    if based_censored_domain not in dic_censored.keys():
        dic_censored[based_censored_domain] = {}
        sub_dic=dic_censored[based_censored_domain]
        sub_dic["blocking rule"]=blocking_rule
        sub_dic["blocking time"]=[]
        
    block_time = dic_censored[based_censored_domain]["blocking time"]
    block_time.append((start, end))


    


# selecting the IP address

In [77]:
dates = generate_dates('2021-06-21','2021-06-30')
print(dates)
country ='CN'
for date in dates:
    print(date)
    filename = '/data/censorship/OONI/' + date +'/'+country+"/combined.csv"
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        df =process_dataframe(df)
        df.to_csv('/data/censorship/OONI/' + date +'/'+country+"/groundtruth_combined.csv")


['2021-06-21', '2021-06-22', '2021-06-23', '2021-06-24', '2021-06-25', '2021-06-26', '2021-06-27', '2021-06-28', '2021-06-29', '2021-06-30']
2021-06-21
starting
 Not matched pattern!
www.childrensdefense.org
 Not matched pattern!
bbs.tianya.cn
 Not matched pattern!
www.iaffe.org
censored !
censored !
 Not matched pattern!
www.naral.org
 Not matched pattern!
www.privacyinternational.org
censored !
censored !
censored !
 Not matched pattern!
www.righttodie.ca
 Not matched pattern!
www.pcusa.org
censored !
 Not matched pattern!
www.fccchina.org
censored !
 Not matched pattern!
www.zensur.freerk.com
 Not matched pattern!
www.youku.com
censored !
 Not matched pattern!
www.cri.cn
censored !
censored !
 Not matched pattern!
wallstreetcn.com
censored !
 Not matched pattern!
www.namecoin.org
censored !
censored !
 Not matched pattern!
ccrjustice.org
 Not matched pattern!
www.wluml.org
 Not matched pattern!
www.dit-inc.us
 Not matched pattern!
www.almanar.com.lb
censored !
 Not matched pattern!


 Not matched pattern!
subscene.com
 Not matched pattern!
www.xinhua.org
 Not matched pattern!
www.fastmail.com
 Not matched pattern!
xiazai.sogou.com
 Not matched pattern!
www.refugeesinternational.org
censored !
 Not matched pattern!
bbs.pku.edu.cn/v2/login.php
censored !
censored !
 Not matched pattern!
www.mormon.org
censored !
 Not matched pattern!
www.bing-amp.com/v0/amp-iframe-0.1.js
 Not matched pattern!
hayoou.com/cx
 Not matched pattern!
letsencrypt.org
 Not matched pattern!
www.ccel.org
censored !
 Not matched pattern!
www.150m.com
 Not matched pattern!
blog.com
 Not matched pattern!
www.socom.mil
censored !
 Not matched pattern!
www.lofter.com
censored !
 Not matched pattern!
blackberry.com
censored !
censored !
 Not matched pattern!
outlook.live.com
 Not matched pattern!
www.vatican.va
censored !
censored !
censored !
 Not matched pattern!
www.microsofttranslator.com
 Not matched pattern!
winscp.net
 Not matched pattern!
video.google.com
 Not matched pattern!
www.iavi.org
 

 Not matched pattern!
tanzil.net
censored !
 Not matched pattern!
www.freehomepage.com
 Not matched pattern!
www.fordfoundation.org/our-work-around-the-world/china
 Not matched pattern!
www.fring.com
 Not matched pattern!
china.usembassy-china.org.cn/zh
 Not matched pattern!
stackexchange.com
censored !
 Not matched pattern!
www.worldwildlife.org
 Not matched pattern!
plus.im
censored !
censored !
censored !
 Not matched pattern!
www.researchgate.net
censored !
 Not matched pattern!
translation2.paralink.com
 Not matched pattern!
www.queernet.org
 Not matched pattern!
www.prolife.com
censored !
 Not matched pattern!
www.socom.mil
 Not matched pattern!
tinyurl.com
 Not matched pattern!
www.fotolog.com
 Not matched pattern!
www.ecequality.org
 Not matched pattern!
www.leaseweb.com
 Not matched pattern!
www.samesexmarriage.ca
censored !
censored !
censored !
 Not matched pattern!
www.msn.com
 Not matched pattern!
www.bitcomet.com
 Not matched pattern!
www.blued.com/en/index.html
censored 

 Not matched pattern!
www.caixinglobal.com/2020-02-03/in-depth-how-wuhan-lost-the-fight-to-contain-the-coronavirus-101510749.html
censored !
 Not matched pattern!
v.qq.com
censored !
 Not matched pattern!
www.womenonweb.org
 Not matched pattern!
www.worldhealth.net
 Not matched pattern!
www.feminist.org
 Not matched pattern!
icao.maps.arcgis.com/apps/opsdashboard/index.html#/977dde48256b489fb48fa98e724721e8
 Not matched pattern!
gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6
censored !
 Not matched pattern!
aidsconcern.org.hk
 Not matched pattern!
www.163.com
 Not matched pattern!
hkrev.info/main.php
 Not matched pattern!
nav.werty.cn
 Not matched pattern!
www.aljazeera.net
 Not matched pattern!
hrntt.org
 Not matched pattern!
www.ahram.org.eg
censored !
 Not matched pattern!
www.fotki.com
 Not matched pattern!
www.transferbigfiles.com
 Not matched pattern!
about.riot.im
 Not matched pattern!
www.aidsonline.com
censored !
 Not matched pattern!

 Not matched pattern!
gpgtools.org
censored !
censored !
censored !
 Not matched pattern!
newbloommag.net
 Not matched pattern!
www.haaretz.com/life/five-films-you-have-to-see-at-tel-aviv-s-international-lgbt-film-festival-1.7338266
censored !
censored !
 Not matched pattern!
www.exgay.com
 Not matched pattern!
wilpf.org
censored !
censored !
 Not matched pattern!
www.globalr2p.org
 Not matched pattern!
www.afp.com
 Not matched pattern!
www.btselem.org
 Not matched pattern!
www.alarabiya.net
censored !
censored !
 Not matched pattern!
www.anonymizer.ru
 Not matched pattern!
www.afterellen.com
 Not matched pattern!
www.derechos.org
 Not matched pattern!
www.almanar.com.lb
censored !
 Not matched pattern!
pflag.org
censored !
censored !
 Not matched pattern!
www.derechos.org
censored !
 Not matched pattern!
www.u4.no
censored !
 Not matched pattern!
www.huffingtonpost.com
 Not matched pattern!
www.hrea.org
 Not matched pattern!
www.radicalparty.org
 Not matched pattern!
www.mideastyouth.

...

 Not matched pattern!
securevpn.im
 Not matched pattern!
www.thestandard.com.hk
 Not matched pattern!
www.glbtqarchive.com
 Not matched pattern!
124.225.213.74/list-motss-1.shtml
 Not matched pattern!
www.getdrupe.com
 Not matched pattern!
retroshare.cc
censored !
 Not matched pattern!
lgbtnewsnow.org
censored !
censored !
 Not matched pattern!
wedo.org
 Not matched pattern!
islamonline.net
censored !
 Not matched pattern!
www.urduvoa.com
 Not matched pattern!
www.well.com/user/queerjhd
 Not matched pattern!
www.mainichi.co.jp
censored !
 Not matched pattern!
www.savethechildren.net
censored !
 Not matched pattern!
freedns.afraid.org
 Not matched pattern!
www.gaywb.com
censored !
 Not matched pattern!
www.acdi-cida.gc.ca
 Not matched pattern!
metasploit.com
 Not matched pattern!
thehackernews.com
 Not matched pattern!
www.weforum.org
 Not matched pattern!
thehiddenwiki.org
 Not matched pattern!
store.steampowered.com
 Not matched pattern!
www.darknet.org.uk
 Not matched pattern!
www.cr

 Not matched pattern!
www.bjd.com.cn
 Not matched pattern!
www.jfdaily.com/home
censored !
 Not matched pattern!
wedo.org
 Not matched pattern!
archive.org/web
 Not matched pattern!
archive.org/web
 Not matched pattern!
www.jhr.ca
censored !
 Not matched pattern!
chinalaborwatch.org/home.aspx
 Not matched pattern!
www.ohchr.org/ch/countries/asiaregion/pages/cnindex.aspx
censored !
 Not matched pattern!
www.savethechildren.net
censored !
 Not matched pattern!
www.piratenpartei.ch
 Not matched pattern!
zuo.la
 Not matched pattern!
www.thegailygrind.com
 Not matched pattern!
www.ectaco.com
 Not matched pattern!
boxmy.hayoou.com/1580562806855/city.html
censored !
 Not matched pattern!
www.ohchr.org/en/countries/asiaregion/pages/cnindex.aspx
censored !
censored !
censored !
censored !
censored !
 Not matched pattern!
www.ifanr.com
 Not matched pattern!
www.fccchina.org
 Not matched pattern!
www.talafone.com
 Not matched pattern!
ccrjustice.org
 Not matched pattern!
www.magicjack.com
 Not ma

 Not matched pattern!
tanzil.net
 Not matched pattern!
reliefweb.int
 Not matched pattern!
map.sogou.com
censored !
 Not matched pattern!
www.ran.org
 Not matched pattern!
www.ran.org
 Not matched pattern!
assets.gitlab-static.net/robots.txt
 Not matched pattern!
bit.ly
censored !
 Not matched pattern!
apps.apple.com/us/developer/apple/id284417353
 Not matched pattern!
www.itsyoursexlife.com
 Not matched pattern!
8.8.8.8/dns-query?dns=q80BAAABAAAAAAAAA3d3dwdleGFtcGxlA2NvbQAAAQAB
 Not matched pattern!
www.blogeasy.com
 Not matched pattern!
www.originalelf.com
 Not matched pattern!
www.enom.com
 Not matched pattern!
kidshealth.org
 Not matched pattern!
exscn.net
 Not matched pattern!
isaalmasih.net
 Not matched pattern!
www.wikia.com/fandom
 Not matched pattern!
www.enom.com
 Not matched pattern!
8.8.4.4/dns-query?dns=q80BAAABAAAAAAAAA3d3dwdleGFtcGxlA2NvbQAAAQAB
 Not matched pattern!
www.christianitytoday.com
 Not matched pattern!
bitly.com
censored !
 Not matched pattern!
tinyurl.com
 N

 Not matched pattern!
massbrowser.cs.umass.edu
censored !
 Not matched pattern!
www.zeit.de
 Not matched pattern!
www.ijm.org
censored !
 Not matched pattern!
www.rescue.org
 Not matched pattern!
secfirst.org
 Not matched pattern!
www.nifty.org
censored !
 Not matched pattern!
lambdalegal.org
censored !
 Not matched pattern!
zuo.la
censored !
 Not matched pattern!
otr.cypherpunks.ca
censored !
censored !
 Not matched pattern!
www.benedelman.org
 Not matched pattern!
www.mormon.org
 Not matched pattern!
news.china.com
 Not matched pattern!
www.ajc.org
 Not matched pattern!
ilga.org
 Not matched pattern!
www.queernet.org
 Not matched pattern!
76crimes.com
 Not matched pattern!
www.postcards-for-iran.org
 Not matched pattern!
i2pforum.net
 Not matched pattern!
www.genderhealth.org
censored !
 Not matched pattern!
github.com/programthink/zhao/issues/38
censored !
 Not matched pattern!
www.unhcr.org
 Not matched pattern!
www.anonymizer.ru
censored !
 Not matched pattern!
www.anonymsurfen.co

 Not matched pattern!
amygoodloe.com/lesbian-dot-org
 Not matched pattern!
ocsp.int-x3.letsencrypt.org
 Not matched pattern!
streetvoice.com
 Not matched pattern!
www.democracycaucus.net
 Not matched pattern!
www.oicc.org
censored !
 Not matched pattern!
www.ushmm.org
 Not matched pattern!
jinshuju.net/f/PAMt4O
 Not matched pattern!
www.americanprogress.org/issues/security/view
 Not matched pattern!
www.ajc.org
censored !
 Not matched pattern!
www.americanprogress.org/issues/security/view
 Not matched pattern!
www.itsyoursexlife.com
 Not matched pattern!
chinaworker.info/en
 Not matched pattern!
lambdalegal.org
censored !
 Not matched pattern!
www.onlinewomeninpolitics.org
 Not matched pattern!
www.opendns.com
censored !
censored !
censored !
censored !
 Not matched pattern!
www.peta.org
 Not matched pattern!
write.as
 Not matched pattern!
www.cseindia.org
 Not matched pattern!
www.vibe.com
 Not matched pattern!
www.gearthblog.com
censored !
 Not matched pattern!
russia.tv
 Not matched